In [195]:
pID = 6
pID = 'sub-' + "%03d" % (pID)

In [196]:
import os

import pandas as pd
import numpy as np

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import AdaBoostClassifier as ABC

from sklearn.model_selection import cross_val_score, KFold

import pickle, json, os
import scipy.io
from bci_funcs import windowed_mean, base_correct, select_mean, slope

# path = 'P:\\Lukas_Gehrke\\fastReach\\data\\eeglab2python'
path = '/Volumes/Lukas_Gehrke/fastReach/data/eeglab2python'

# Preconscious Augmentation

This script trains the classifier for a brain-computer interface that controls electrical muscle stimulation in the preconscious augmentation experiment.
The functions used to build the feature vectors are the same that are used for the online application and are found in 'bci_funcs'

A two class linear discriminant model is fitted to idle and pre-movement EEG training data. The model and a channel selection is saved.

In [197]:
# load data
pre_move = scipy.io.loadmat(path+os.sep+pID+os.sep+'pre_move_Baseline.mat')
idle = scipy.io.loadmat(path+os.sep+pID+os.sep+'idle_Baseline.mat')
all_idle = idle['idle']
all_pre_move = pre_move['pre_move']

## Select Best Channels by Slope

In [198]:
# slope_idle = np.zeros((idle.shape[2], idle.shape[0]))
# slope_pre_move = np.zeros((pre_move.shape[2], pre_move.shape[0]))

# for trial_ix in range(0, pre_move.shape[2]):
#     slope_pre_move[trial_ix, :] = slope(pre_move[:,:,trial_ix], 'linear').flatten()

# for trial_ix in range(0, idle.shape[2]):
#     slope_idle[trial_ix, :] = slope(idle[:,:,trial_ix], 'linear').flatten()

# crit1 = pd.DataFrame(slope_pre_move.mean(axis=0)).sort_values(by=0, ascending=False)
# crit1['chans'] = crit1.index
# crit1['crit1_ranks'] = crit1.reset_index().index
# crit1.sort_values(by='chans', inplace=True)

# crit2 = pd.DataFrame(slope_idle.mean(axis=0)).sort_values(by=0, ascending=True)
# crit2['chans'] = crit2.index
# crit2['crit2_ranks'] = crit2.reset_index().index
# crit2.sort_values(by='chans', inplace=True)

# crits = pd.concat([crit1.chans, crit1.crit1_ranks, crit2.crit2_ranks], axis=1)
# crits['mean_ranks'] = crits[['crit1_ranks', 'crit2_ranks']].mean(axis=1)
# crits.sort_values(by='mean_ranks', inplace=True)
# chans = crits.chans.values

# chans = chans[:nr_chans]
# chan_names.Var1[chans]

## Compute Features

In [199]:
# data = slope

# data_to_plot = pd.DataFrame(data)
# data_to_plot['class'] = classes
# data_to_plot = data_to_plot.melt(id_vars = 'class', value_name = 'amplitude')

# import seaborn as sns
# import matplotlib.pyplot as plt

# sns.set_theme(style="ticks")
# sns.displot(x="amplitude", hue="class", data=data_to_plot, kde=True, fill=True, stat="density")


In [ ]:
# from lazypredict.Supervised import LazyClassifier
# from sklearn.model_selection import train_test_split

# for i in range(1, 10):
#     X_train, X_test, y_train, y_test = train_test_split(np.asarray(data), classes, test_size=.1)#,random_state =123)

#     clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
#     models,predictions = clf.fit(X_train, X_test, y_train, y_test)
#     print(models)

# Compute Features & Model fitting

In [200]:
# ### COPY IN WHERE BEST CLASSIFICATION PERFORMANCE WAS ACHIEVED ###
# filename = path+os.sep+pID+os.sep+'model_'+pID+'_eeg.sav'
# pickle.dump(clf, open(filename, 'wb'))

# threshold = cv_results.mean()
# bci_params = dict(((k, eval(k)) for k in ('chans', 'windows', 'baseline', 'target_class', 'threshold', 'data_srate', 'classifier_update_rate')))
# with open(path+os.sep+pID+os.sep+'bci_params.json', 'w') as f:
#     json.dump(bci_params, f)
# ### END ###

nr_chans_arr = np.arange(10,21,5) # select one and then save the model

data_srate = 250
windows = 10
baseline = data_srate/windows
target_class = 1.0
classifier_update_rate = 5 # samples

chan_names = pd.read_csv(path+os.sep+pID+os.sep+'sel_chans_names.csv')
chans = pd.read_csv(path+os.sep+pID+os.sep+'sel_chans.csv', header=None)

feat = 'slope' # slope or slope_2d

for nr_chans in nr_chans_arr:
    print(str(nr_chans) + ' channels')

    chans = np.squeeze(np.array(chans[0:nr_chans]) - 1).tolist()
    # chan_names.Var1[chans]

    idle = all_idle[chans, :, :]
    pre_move = all_pre_move[chans, :, :]

    if feat == 'slope':
        idle_feat = np.zeros((idle.shape[2], idle.shape[0]))
        pre_move_feat = np.zeros((pre_move.shape[2], pre_move.shape[0]))

    elif feat == 'slope2d':
        idle_feat = np.zeros((idle.shape[2], idle.shape[0]*2))
        pre_move_feat = np.zeros((pre_move.shape[2], pre_move.shape[0]*2))

    elif feat == 'windowed_means':
        idle_feat = np.zeros((idle.shape[2], idle.shape[0] * (windows)))
        pre_move_feat = np.zeros((pre_move.shape[2], pre_move.shape[0] * (windows)))

    for trial_ix in range(0, pre_move.shape[2]):
        if feat == 'slope':
            pre_move_feat[trial_ix, :] = slope(pre_move[:,:,trial_ix], 'linear').flatten()
        
        elif feat == 'slope2d':
            pre_move_feat[trial_ix, :idle.shape[0]] = slope(pre_move[:,:150,trial_ix], 'linear').flatten()
            pre_move_feat[trial_ix, idle.shape[0]:] = slope(pre_move[:,150:,trial_ix], 'linear').flatten()

        elif feat == 'windowed_means':
            tmp = base_correct(pre_move[:,:,trial_ix], baseline-1)
            pre_move_feat[trial_ix, :] = windowed_mean(tmp, windows).flatten()

    for trial_ix in range(0, idle.shape[2]):
        if feat == 'slope':
            idle_feat[trial_ix, :] = slope(idle[:,:,trial_ix], 'linear').flatten()    
        
        elif feat == 'slope2d':
            idle_feat[trial_ix, :idle.shape[0]] = slope(idle[:,:150,trial_ix], 'linear').flatten()
            idle_feat[trial_ix, idle.shape[0]:] = slope(idle[:,150:,trial_ix], 'linear').flatten()

        elif feat == 'windowed_means':
            tmp = base_correct(idle[:,:,trial_ix], baseline-1)
            idle_feat[trial_ix, :] = windowed_mean(tmp, windows).flatten()

    data = np.concatenate((pre_move_feat, idle_feat), axis = 0)
    pre_move_class = np.ones((pre_move_feat.shape[0], 1))
    idle_class = np.zeros((idle_feat.shape[0], 1))
    classes = np.concatenate((pre_move_class, idle_class)).ravel()

    clf = LDA(solver='eigen', shrinkage='auto')
    clf.fit(data, classes)
    kfolds = KFold(n_splits=5, random_state=1, shuffle=True) 
    cv_results = cross_val_score(clf, data, classes, cv=kfolds)
    print("LDA accuracy: " + str(cv_results.mean()) + " +/- " + str(cv_results.std()))

    clf = ABC()
    clf.fit(data, classes)
    kfolds = KFold(n_splits=5, random_state=1, shuffle=True) 
    cv_results = cross_val_score(clf, data, classes, cv=kfolds)
    print("ABC accuracy: " + str(cv_results.mean()) + " +/- " + str(cv_results.std()))

    clf = QDA()
    clf.fit(data, classes)
    kfolds = KFold(n_splits=5, random_state=1, shuffle=True) 
    cv_results = cross_val_score(clf, data, classes, cv=kfolds)
    print("QDA accuracy: " + str(cv_results.mean()) + " +/- " + str(cv_results.std()))

    clf = RFC()
    clf.fit(data, classes)
    kfolds = KFold(n_splits=5, random_state=1, shuffle=True) 
    cv_results = cross_val_score(clf, data, classes, cv=kfolds)
    print("RFC accuracy: " + str(cv_results.mean()) + " +/- " + str(cv_results.std()))

10 channels
LDA accuracy: 0.6404761904761905 +/- 0.049026810195176206
ABC accuracy: 0.6843915343915344 +/- 0.080527640448701
QDA accuracy: 0.7492063492063492 +/- 0.09191970161216605
RFC accuracy: 0.7275132275132276 +/- 0.07849166671024589
15 channels
LDA accuracy: 0.6119047619047618 +/- 0.038095238095238106
ABC accuracy: 0.641005291005291 +/- 0.08820520013254253
QDA accuracy: 0.6984126984126984 +/- 0.0717365494506455
RFC accuracy: 0.6915343915343916 +/- 0.08127507699184096
20 channels
LDA accuracy: 0.6042328042328042 +/- 0.05084185323058649
ABC accuracy: 0.6978835978835979 +/- 0.0971969336869399
QDA accuracy: 0.647883597883598 +/- 0.04461732564060183
RFC accuracy: 0.7407407407407407 +/- 0.07716981551150617
